https://plotly.com/

In [129]:
# Imports
import random
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime

## Carregando e Visualizando os Dados com Candlestick

In [130]:
# Carrega os dados
df = pd.read_csv('dataset.csv')

In [131]:
df.shape

(506, 11)

In [132]:
df.head()

,Date,AAPL.Open,AAPL.High,AAPL.Low,AAPL.Close,AAPL.Volume,AAPL.Adjusted,dn,mavg,up,direction
0,2015-02-17,127.489998,128.880005,126.919998,127.830002,63152400,122.905254,106.741052,117.927667,129.114281,Increasing
1,2015-02-18,127.629997,128.779999,127.449997,128.720001,44891700,123.760965,107.842423,118.940333,130.038244,Increasing
2,2015-02-19,128.479996,129.029999,128.330002,128.449997,37362400,123.501363,108.894245,119.889167,130.884089,Decreasing
3,2015-02-20,128.619995,129.500000,128.050003,129.500000,48948400,124.510914,109.785449,120.763500,131.741551,Increasing
4,2015-02-23,130.020004,133.000000,129.660004,133.000000,70974100,127.876074,110.372516,121.720167,133.067817,Increasing


O Candlestick é um tipo de gráfico financeiro utilizado para descrever a movimentação de preços de determinados ativos (ações, criptomoedas, etc.). Em contraste com um gráfico de linha simples do preço de fechamento, ele oferece muito mais informações sobre a dinâmica dos preços - é baseado em dados OHLC, o que significa que contém os preços de abertura, alta, baixa e fechamento (geralmente junto com o volume). Esses valores podem ser mostrados em diferentes frequências de amostragem (minuto, hora, dia, semana, etc.) e são frequentemente usados como base para análises técnicas.

https://plotly.com/python/candlestick-charts/

In [133]:
# Candlestick
fig = go.Figure(data = [go.Candlestick(x = df['Date'],
                open = df['AAPL.Open'],
                high = df['AAPL.High'],
                low = df['AAPL.Low'],
                close = df['AAPL.Close'])])

fig.show()

In [134]:
# Vamos trabalhar com a cotação de fechamento da ação da Apple
precos = df['AAPL.Close'].values

In [135]:
type(precos)

numpy.ndarray

## Configuração do Algoritmo Q-Learning

In [136]:
# Configuração do Q-Learning
print('\nDefinindo os Hiperparâmetros do Q-Learning...')
num_episodios = 1000
alfa = 0.1
gama = 0.99
epsilon = 0.1


Definindo os Hiperparâmetros do Q-Learning...


## Configuração do Ambiente de Negociação

In [137]:
# Configuração do ambiente de negociação
print('\nConfigurando o Ambiente de Negociação...')
acoes = ['comprar', 'vender', 'manter']
saldo_inicial = 1000
num_acoes_inicial = 0


Configurando o Ambiente de Negociação...


## Função Para Executar as Ações (Passos) do Robô Trading

In [138]:
# Função para executar uma ação e retornar a recompensa e o próximo estado
def executar_acao(estado, acao, saldo, num_acoes, preco):
    
    # Ação de comprar
    if acao == 0:  
        if saldo >= preco:
            num_acoes += 1
            saldo -= preco
    
    # Ação de vender
    elif acao == 1:  
        if num_acoes > 0:
            num_acoes -= 1
            saldo += preco

    # Define o lucro
    lucro = saldo + num_acoes * preco - saldo_inicial

    return (saldo, num_acoes, lucro)

## Treinamento do Robô

In [139]:
# Inicializar a tabela Q
print('\nInicializando a Tabela Q...')
q_tabela = np.zeros((len(precos), len(acoes)))


Inicializando a Tabela Q...


In [140]:
# Treinamento
print('\nInicializando o Treinamento...')
for _ in range(num_episodios):
    
    # Define o saldo
    saldo = saldo_inicial
    
    # Define o número de ações
    num_acoes = num_acoes_inicial

    # Loop
    for i, preco in enumerate(precos[:-1]):
        
        estado = i

        # Escolher a ação usando a política epsilon-greedy
        if np.random.random() < epsilon:
            acao = random.choice(range(len(acoes)))
        else:
            acao = np.argmax(q_tabela[estado])

        # Executar a ação e obter a recompensa e o próximo estado
        saldo, num_acoes, lucro = executar_acao(estado, acao, saldo, num_acoes, preco)
        prox_estado = i + 1

        # Atualizar a tabela Q
        q_tabela[estado][acao] += alfa * (lucro + gama * np.max(q_tabela[prox_estado]) - q_tabela[estado][acao])

print('\nTreinamento Concluído...')


Inicializando o Treinamento...

Treinamento Concluído...


## Executando o Robô Trading e Prevendo o Lucro de Operações na Bolsa de Valores

In [141]:
# Valores iniciais para testar o agente
saldo = saldo_inicial
num_acoes = num_acoes_inicial

In [142]:
print('\nExecutando o Agente...')
for i, preco in enumerate(precos[:-1]):
    estado = i
    acao = np.argmax(q_tabela[estado])
    print(str(estado) + ' | ' + str(acao) + ' | ' + str(saldo) + ' | ' + str(num_acoes) + ' | ' + str(preco))
    saldo, num_acoes, _ = executar_acao(estado, acao, saldo, num_acoes, preco)
    
print('\nExecução Concluída...')


Executando o Agente...
0 | 1 | 1000 | 0 | 127.830002
1 | 0 | 1000 | 0 | 128.720001
2 | 0 | 871.279999 | 1 | 128.449997
3 | 0 | 742.8300019999999 | 2 | 129.5
4 | 1 | 613.3300019999999 | 3 | 133.0
5 | 0 | 746.3300019999999 | 2 | 132.169998
6 | 0 | 614.160004 | 3 | 128.789993
7 | 1 | 485.370011 | 4 | 130.419998
8 | 1 | 615.7900089999999 | 3 | 128.460007
9 | 0 | 744.250016 | 2 | 129.089996
10 | 1 | 615.1600199999999 | 3 | 129.360001
11 | 2 | 744.5200209999999 | 2 | 128.539993
12 | 0 | 744.5200209999999 | 2 | 126.410004
13 | 1 | 618.110017 | 3 | 126.599998
14 | 1 | 744.710015 | 2 | 127.139999
15 | 0 | 871.850014 | 1 | 124.510002
16 | 2 | 747.340012 | 2 | 122.239998
17 | 2 | 747.340012 | 2 | 124.449997
18 | 0 | 747.340012 | 2 | 123.589996
19 | 0 | 623.750016 | 3 | 124.949997
20 | 0 | 498.80001899999996 | 4 | 127.040001
21 | 1 | 371.76001799999995 | 5 | 128.470001
22 | 0 | 500.23001899999997 | 4 | 127.5
23 | 2 | 372.73001899999997 | 5 | 125.900002
24 | 1 | 372.73001899999997 | 5 | 127.209999

In [143]:
num_acoes

7

In [144]:
precos[-1]

135.350006

In [145]:
# Vendendo todas as ações no último preço
saldo += num_acoes * precos[-1]
lucro = saldo - saldo_inicial
lucro_final = round(lucro,2)

In [146]:
print(f"\nComeçamos a Negociação com Saldo Inicial de 1000 e Tivemos Lucro de: {lucro_final}")


Começamos a Negociação com Saldo Inicial de 1000 e Tivemos Lucro de: 123.51


# Fim